In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Read the Excel file
df = pd.read_excel('your_excel_file.xlsx')

# Extract node labels
labels = df.iloc[:, 0].tolist()

# Assuming your adjacency matrix is in the remaining columns
adjacency_matrix = df.iloc[:, 1:].values

# Create a graph from the adjacency matrix
G = nx.from_numpy_matrix(adjacency_matrix)

# Define positions of nodes (you might have your own positions)
pos = nx.spring_layout(G, dim=3)

# Extract node positions for Plotly
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_z = [pos[node][2] for node in G.nodes()]

# Create edges
edges_x = []
edges_y = []
edges_z = []

for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    edges_z.extend([z0, z1, None])

# Create nodes trace
nodes_trace = go.Scatter3d(
    x=node_x,
    y=node_y,
    z=node_z,
    mode='markers',
    text=labels,  # Use labels as hover text
    marker=dict(size=6, color='blue')
)

# Create edges trace
edges_trace = go.Scatter3d(
    x=edges_x,
    y=edges_y,
    z=edges_z,
    mode='lines',
    line=dict(color='red', width=2)
)

# Create the figure
fig = go.Figure(data=[nodes_trace, edges_trace])

# Set layout options
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1])
    )
)

# Show the figure
fig.show()


In [9]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Read the CSV file, assuming the first row contains headers (node names)
df = pd.read_csv('/content/HPRD_PPI.csv', header=0, index_col=0)

# Extract node labels
labels = df.columns.tolist()

# Drop the first row (which contains node names)
df = df.iloc[1:]

# Assuming your adjacency matrix is in the remaining rows
adjacency_matrix = df.values

# Create a graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix)

# Define positions of nodes (you might have your own positions)
pos = nx.spring_layout(G, dim=3)

# Extract node positions for Plotly
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_z = [pos[node][2] for node in G.nodes()]

# Create edges
edges_x = []
edges_y = []
edges_z = []

for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    edges_z.extend([z0, z1, None])

# Create nodes trace
nodes_trace = go.Scatter3d(
    x=node_x,
    y=node_y,
    z=node_z,
    mode='markers',
    text=labels,  # Use labels as hover text
    marker=dict(size=6, color='blue'),
    hovertemplate='%{text}<extra></extra>'
)

# Create edges trace
edges_trace = go.Scatter3d(
    x=edges_x,
    y=edges_y,
    z=edges_z,
    mode='lines',
    line=dict(color='red', width=2)
)

# Create the figure
fig = go.Figure(data=[nodes_trace, edges_trace])

# Set layout options
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1])
    ),
    hovermode='closest'
)

# Show the figure
fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx
from ipywidgets import interact, widgets

# Read the CSV file, assuming the first row contains headers (node names)
df = pd.read_csv('/content/HPRD_PPI.csv', header=0, index_col=0)

# Extract node labels
labels = df.columns.tolist()

# Drop the first row (which contains node names)
df = df.iloc[1:]

# Assuming your adjacency matrix is in the remaining rows
adjacency_matrix = df.values

# Create a graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix)

# Define positions of nodes (you might have your own positions)
pos = nx.spring_layout(G, dim=3)

# Extract node values (assumed to be in a column named 'values')
node_values = df['values'].tolist()  # Replace 'values' with your actual column name

# Define a color scale for node sizes (e.g., using a logarithmic scale)
node_sizes = [val * 20 for val in node_values]  # Adjust the scaling factor as needed

# Define a color scale for node colors (light to dark)
color_scale = ['rgb(255,255,255)', 'rgb(0,0,0)']  # Light gray to black

# Normalize node values for color scale
normalized_values = [(val - min(node_values)) / (max(node_values) - min(node_values)) for val in node_values]

# Map normalized values to colors using the color scale
node_colors = [color_scale[int(val * (len(color_scale) - 1))] for val in normalized_values]

# Function to update the figure based on selected nodes
def update_network(selected_nodes):
    if not selected_nodes:  # Check if the input is empty
        selected_nodes = labels  # If empty, use all nodes
    else:
        selected_nodes = selected_nodes.split(',')
        selected_nodes = [node.strip() for node in selected_nodes if node.strip() in labels]


    # Create nodes trace for selected nodes
    selected_nodes_trace = go.Scatter3d(
        x=[pos[node][0] for node in selected_nodes],
        y=[pos[node][1] for node in selected_nodes],
        z=[pos[node][2] for node in selected_nodes],
        mode='markers',
        text=selected_nodes,  # Use labels as hover text
        marker=dict(size=[node_sizes[labels.index(node)] for node in selected_nodes], color=node_colors),
        hovertemplate='%{text}<extra></extra>'
    )

    # Create edges
    edges_x = []
    edges_y = []
    edges_z = []

    for edge in G.edges():
        if edge[0] in selected_nodes and edge[1] in selected_nodes:
            x0, y0, z0 = pos[edge[0]]
            x1, y1, z1 = pos[edge[1]]
            edges_x.extend([x0, x1, None])
            edges_y.extend([y0, y1, None])
            edges_z.extend([z0, z1, None])

    # Create edges trace
    edges_trace = go.Scatter3d(
        x=edges_x,
        y=edges_y,
        z=edges_z,
        mode='lines',
        line=dict(color='red', width=2)
    )

    # Create the figure
    fig = go.Figure(data=[selected_nodes_trace, edges_trace])

    # Set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=4, range=[-1, 1]),
            yaxis=dict(nticks=4, range=[-1, 1]),
            zaxis=dict(nticks=4, range=[-1, 1])
        ),
        hovermode='closest'
    )

    # Show the figure
    fig.show()

# Create a text box widget for entering nodes
node_input = widgets.Text(
    value='',
    placeholder='Enter nodes (comma-separated)',
    description='Nodes:',
    disabled=False
)

# Define an interaction using the widget
interact(update_network, selected_nodes=node_input);


In [6]:
labels = df.columns.tolist()
labels

['EEF1A1',
 'TPT1',
 'RPL23A',
 'UBC',
 'RPS2',
 'ACTB',
 'HUWE1',
 'RPS4X',
 'RPLP0',
 'RPS10',
 'RPL7',
 'PPIA',
 'FTL',
 'GAPDH',
 'RPS15A',
 'RPLP1',
 'RPS15',
 'CFL1',
 'RPL31',
 'TMSB10',
 'UBB',
 'RPS14',
 'RPL30',
 'B2M',
 'RPS6',
 'RPS19',
 'RPL12',
 'HLA-B',
 'RPS16',
 'RPL3',
 'RPL9',
 'HLA-C',
 'HLA-A',
 'TUBA1B',
 'RPS21',
 'RPL24',
 'RPL14',
 'RPL38',
 'PABPC1',
 'COL1A2',
 'FTH1',
 'RPS3',
 'ANXA2',
 'TUBA1C',
 'RPL8',
 'RPL22',
 'HSP90AA1',
 'RPL29',
 'OAZ1',
 'COX6C',
 'FN1',
 'CD24',
 'EIF1',
 'RPS7',
 'COL3A1',
 'MGP',
 'PSAP',
 'HMGB1',
 'EEF2',
 'RPS27A',
 'RPL11',
 'RPL6',
 'TMSB4X',
 'VIM',
 'ALDOA',
 'COL1A1',
 'HSP90AB1',
 'UBA52',
 'LDHA',
 'LAMP1',
 'RPL15',
 'PTMA',
 'HMGN2',
 'LGALS1',
 'RPL35A',
 'MYL6',
 'MIF',
 'EEF1D',
 'NPM1',
 'FAU',
 'CD63',
 'DDX17',
 'RPS9',
 'SKP1',
 'IGLC1',
 'BTF3',
 'DSTN',
 'TUBB',
 'MORF4L2',
 'CD74',
 'GATA3',
 'YWHAQ',
 'CLTC',
 'KRT18',
 'YWHAB',
 'ATP5A1',
 'JTB',
 'RHOA',
 'HSPD1',
 'HLA-G',
 'SPARC',
 'PRDX1',
 'SRP14',

In [10]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

# Read the CSV file, assuming the first row contains headers (node names)
df = pd.read_csv('/content/HPRD_PPI.csv', header=0, index_col=0)

# Extract node labels
labels = df.columns.tolist()

# Drop the first row (which contains node names)
df = df.iloc[1:]

# Assuming your adjacency matrix is in the remaining rows
adjacency_matrix = df.values

# Create a graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix)

# Define positions of nodes using a combination of Kamada-Kawai and Circular layout
pos_kk = nx.kamada_kawai_layout(G, dim=3)
pos_circular = nx.circular_layout(G, dim=3)

# Interpolate between the two layouts to create a circular layout in 3D space
interpolated_pos = {node: 0.5 * (pos_kk[node] + pos_circular[node]) for node in G.nodes()}

# Extract node positions for Plotly
node_x = [pos[0] for pos in interpolated_pos.values()]
node_y = [pos[1] for pos in interpolated_pos.values()]
node_z = [pos[2] for pos in interpolated_pos.values()]

# Create edges
edges_x = []
edges_y = []
edges_z = []

for edge in G.edges():
    x0, y0, z0 = interpolated_pos[edge[0]]
    x1, y1, z1 = interpolated_pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    edges_z.extend([z0, z1, None])

# Create nodes trace
nodes_trace = go.Scatter3d(
    x=node_x,
    y=node_y,
    z=node_z,
    mode='markers',
    text=labels,  # Use labels as hover text
    marker=dict(size=6, color='blue')
)

# Create edges trace
edges_trace = go.Scatter3d(
    x=edges_x,
    y=edges_y,
    z=edges_z,
    mode='lines',
    line=dict(color='red', width=2)
)

# Create the figure
fig = go.Figure(data=[nodes_trace, edges_trace])

# Set layout options
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    )
)

# Show the figure
fig.show()


KeyboardInterrupt: ignored